<a href="https://colab.research.google.com/github/nomi181472/MS-DS-Practice/blob/main/cnn/VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import  torch
import torch.nn as nn

In [2]:
architecture={
    "VGG11":[
        1,1,1,2,2,
    ],
    "VGG13":[
        2,2,2,2,2,
    ],
    "VGG16":[
        2,2,3,3,3
    ],
    "VGG19":[
        2,2,4,4,4
    ]

}

In [35]:
class Block(nn.Module):
  def __init__(self,in_channels,out_channels,no_layers):
    super().__init__()
    self.relu=nn.ReLU()
    layers=[
        nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=3,stride=1,padding=1,),
        self.relu
    ]
    for _ in range(no_layers-1):
      layers.append(
           nn.Conv2d(in_channels=out_channels,out_channels=out_channels,kernel_size=3,stride=1,padding=1,),
      )
      layers.append(self.relu)
    layers.append(
        nn.MaxPool2d(kernel_size=2,stride=2)
    )
    self.conv_block=nn.Sequential(*layers)
  def forward(self,x):
    return self.conv_block(x)
class VGG(nn.Module):
  def __init__(self,in_channels,classes,VGG_type):
    super(VGG,self).__init__()
    self.feature_maps=[in_channels,64,128,256,512,512]
    self.convs=nn.ModuleList([])
    for i in range(len(VGG_type)):
      self.convs.append(
          Block(self.feature_maps[i],self.feature_maps[i+1],VGG_type[i])
      )

    self.fc1=nn.Linear(7*7*512,4096)
    self.fc2=nn.Linear(4096,4096)
    self.fc3=nn.Linear(4096,1000)

    self.dropout=nn.Dropout(0.5)
    self.relu=nn.ReLU()

    self.init_weight()
  def forward(self,x):
    for block in self.convs:
      x=block(x)

    x=torch.flatten(x,1)
    x=self.relu(self.dropout(self.fc1(x)))
    x=self.relu(self.dropout(self.fc2(x)))
    x=self.fc3(x)

    return x
  def init_weight(self,):
    for layers in self.modules():
      if isinstance(layers,nn.Conv2d) or isinstance(layers,nn.Linear):
        nn.init.normal_(layers.weight,std=0.01)
        nn.init.constant_(layers.bias,0)




vgg=VGG(3,1000,architecture["VGG16"])
print(next(iter(vgg.modules())))
vgg(torch.rand(16,3,224,224))

VGG(
  (convs): ModuleList(
    (0): Block(
      (relu): ReLU()
      (conv_block): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU()
        (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (1): Block(
      (relu): ReLU()
      (conv_block): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
        (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU()
        (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (2): Block(
      (relu): ReLU()
      (conv_block): Sequential(
        (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
        (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1

tensor([[-7.8421e-08,  3.7633e-08, -1.1618e-07,  ...,  1.0689e-07,
          3.1656e-08, -2.4136e-08],
        [-7.4036e-08, -8.1031e-08, -7.4207e-08,  ...,  9.3238e-08,
          2.9974e-09, -2.7592e-08],
        [-3.3393e-08, -1.1178e-07, -7.9570e-08,  ...,  1.0025e-07,
          4.2111e-08, -1.1340e-08],
        ...,
        [-4.9779e-08, -2.2393e-08, -5.0897e-08,  ...,  5.3535e-08,
         -2.3452e-08, -6.1374e-08],
        [ 1.5143e-08, -8.3182e-08, -8.4527e-08,  ...,  6.9509e-08,
          3.9132e-08, -3.7813e-09],
        [-3.4156e-08, -5.2256e-08, -1.0575e-08,  ...,  9.6758e-08,
         -1.7078e-08, -1.9949e-08]], grad_fn=<AddmmBackward0>)

<ipython-input-25-da4dbeb7572e>:31: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(layers.bias,0)


<generator object Module.modules at 0x798a44299770>


RuntimeError: Given input size: (512x1x1). Calculated output size: (512x0x0). Output size is too small